In [ ]:
!pip install mlxtend --quiet

In [ ]:

import pandas as pd
import matplotlib.pyplot as plt
from mlxtend.frequent_patterns import apriori, association_rules
from mlxtend.preprocessing import TransactionEncoder
from google.colab import files

In [ ]:
# ---------------- LOAD DATA ----------------
dataset = pd.read_csv("/content/Market_Basket_Optimisation.csv", header=None)
dataset.head()

In [ ]:
# ---------------- DATA PREPROCESSING ----------------
transactions = []
for i in range(dataset.shape[0]):
    row_items = []
    for j in range(dataset.shape[1]):
        value = str(dataset.values[i, j]).strip()
        if value != 'nan' and value != "":
            row_items.append(value)
    transactions.append(row_items)

print("Number of transactions:", len(transactions))
print("Example transaction:", transactions[0])


In [ ]:
# ---------------- TRANSACTION ENCODING ----------------
te = TransactionEncoder()
te_array = te.fit(transactions).transform(transactions)
df = pd.DataFrame(te_array, columns=te.columns_)
df.head()

In [ ]:
# ---------------- APRIORI ALGORITHM ----------------
frequent_itemsets = apriori(df, min_support=0.003, use_colnames=True)
print("Number of frequent itemsets:", len(frequent_itemsets))
frequent_itemsets.head()

In [ ]:
# ---------------- ASSOCIATION RULES ----------------
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)
rules = rules.sort_values(by="lift", ascending=False)
rules.head(10)


In [ ]:
# ---------------- VISUALIZATION ----------------
plt.figure(figsize=(10,6))
plt.scatter(rules['support'], rules['confidence'],
            s=rules['lift']*10,
            c=rules['lift'],
            cmap='viridis',
            alpha=0.7)
plt.xlabel("Support")
plt.ylabel("Confidence")
plt.title("Association Rules (Support vs Confidence)")
plt.colorbar(label='Lift')
plt.show()


In [ ]:
rules_strict = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.4)
rules_strict = rules_strict.sort_values(by="lift", ascending=False)

print("Original rules:", len(rules))
print("Rules after increasing confidence:", len(rules_strict))

In [ ]:
rules_strict.to_csv("association_rules_output.csv", index=False)
files.download("association_rules_output.csv")